# CSCE-629: Analysis of Algorithms - Project

##### Author - Shabarish Kumar Rajendra Prasad

## Creating the Graph Object
The first step in the approach is that we need to model our Graph data-structure. I am using the defaultdict module from python's collections as a hashmap for adjacency lists of the vertices. The graph object also provides methods to insert edges, get neighbors and get edges of a particular vertex

In [0]:
import numpy as np
from collections import defaultdict, deque
import random
import time

In [0]:
class Graph:

  def __init__(self):

    #initialize adjacency list as a dictionary of lists
    self.adjList = defaultdict(list)

    #initialize neighbors as a dictionary of sets
    self.neighbor = defaultdict(set)

    #initialize edge list
    self.edgeList = []
  
  def InsertEdge(self, u, v, w):

    #prevent duplicates and add edge for u
    if v not in self.neighbor[u]:
      self.neighbor[u].add(v)
      self.adjList[u].append([v,w])
    
    #prevent duplicates and add edge for v
    if v not in self.neighbor[v]:
      self.neighbor[v].add(u)
      self.adjList[v].append([u,w])
    
    #append edge to edgeList
    self.edgeList.append([w,u,v])

  def getNeighbors(self, v):

    #From neighbors dictionary return the element
    return self.neighbor[v]

  def getEdges(self, v):
    
    #From adjacency list dictionary return the element
    return self.adjList[v]

  def getDegree(self, v):

    #The length of the adjacency list will the degree of the element
    return len(self.adjList[v])
  
  def getEdgeList(self):

    #return the edge list
    return self.edgeList

## 1. Random Graph Generation


### 1.1 Sparse Graph
The projects requires us to create two graphs, the first of which is a sparse graph. We first make sure we have a fully connceted graph and therefore we randomnly shuffle the numbers between 0-4999 and make edges between the adjacent vertices in the shuffled list. Then we go on and create the graph such that it has 15000 edges. Since there are 15000 undirected edges and 5000 vertices in our graph, the average degree of the vertices is exacty 6.

In [0]:
def generateSparseGraph():  
  
  #Initialize the Sparse Graph
  SparseGraph = Graph()

  #Making sure we have a fully-connected Graph
  #List all 5000 vertices and randomly shuffle the list
  stack = list(range(5000))
  random.shuffle(stack)

  #Insert edges between all the neighbors in the list
  previous = stack.pop()
  while(True):
    current = stack.pop()
    weight = random.randint(1,999)
    SparseGraph.InsertEdge(previous,current,weight)
    previous = current

    #break if the stack is empty
    if not stack:
      break

  #Create more edges so that the average degree of the vertices is 6
  for _ in range(10001):
    while(True):
      u = random.randint(0,4999)
      v = random.randint(0,4999)

      if u != v and v not in SparseGraph.getNeighbors(u):
        weight = random.randint(1,999)
        SparseGraph.InsertEdge(u,v,weight)
        break
      
      else:
        continue
  return SparseGraph

### 1.2 Dense Graph

Create the second Graph, which is the dense graph, in which every vertex is connected with 20% of the graph. We also make sure that the graph is fully-connected by following the steps followed for the sparse graph. Then we connect each vertex to about 20% of the graph.

In [0]:
def generateDenseGraph():
    
  #Initialize the Dense Graph
  DenseGraph = Graph()

  #Making sure we have a fully-connected Graph
  #List all 5000 vertices and randomly shuffle the list
  stack = list(range(5000))
  random.shuffle(stack)

  #Insert edges between all the neighbors in the list
  previous = stack.pop()
  while(True):
    current = stack.pop()
    weight = random.randint(1,999999)
    DenseGraph.InsertEdge(previous,current,weight)
    previous = current

    #break if the stack is empty
    if not stack:
      break

  available = list(range(5000))

  #Create more edges so that every vertex is connected to about  20% of the graph
  for u in range(5000):
    while(True):
      no_edges = len(DenseGraph.getNeighbors(u))

      #break if the vertex is connected to 20% of the graph
      if no_edges > 1000:
        break

      while(True):
        v = random.choice(available)

        if DenseGraph.getDegree(v)>=1025:
          available.remove(v)
          continue

        if u != v and v not in DenseGraph.getNeighbors(u):
          weight = random.randint(1,999999)
          DenseGraph.InsertEdge(u,v,weight)
          break
        
        else:
          continue
  return DenseGraph

## 2. Heap Structure

The next part of the project is to define a maximum heap structure. We have created a heap structure here which creates uses the first element in a tuple for comparison. Hence whenever we create a list to insert in the heap, we should make sure that the element to be comapred is in the index 0 (first element) of the tuple

In [0]:
class MaxHeap:

  def __init__(self):

    #initialize the array when the object gets initialized
    self.heap = []

  def push(self, value):

    #add the pushed value as the last element of the array
    self.heap.append(value)

    #Keep pushing the element above until it is lesser than its parent
    i = int(len(self.heap))-1
    while ((i>0) and (self.heap[i//2]<self.heap[i])):
      self.heap[i//2], self.heap[i] = self.heap[i], self.heap[i//2]
      i = i//2

  def pop(self):

    #Temporily store the max element
    ret = self.heap[0]

    #delete the element from the heap
    self.delete(0)

    #return element
    return ret

  def delete(self, i):
    #get length of the heap
    n = len(self.heap)

    #Check corner cases
    if n==1 and i == 0:
      self.heap.pop()

    else:
      #Swap the last element to the element to be replaced
      self.heap[i] = self.heap.pop()
      n = n-1

      #Shift up if applicable
      if (i>0) and (self.heap[i/2] < self.heap[i]):

        #Keep shifting until the heap is proper
        while ((i>0) and (self.heap[i/2] < self.heap[i])):
          self.heap[i/2], self.heap[i] = self.heap[i], self.heap[i/2]
          i = i/2
      
      #Else shift down
      else:

          #Keep shifting until the heap is proper
          while(((2*i < n) and (self.heap[2*i] > self.heap[i])) or 
                (((2*i)+1 < n) and (self.heap[(2*i)+1] > self.heap[i]))):
            
            #Check which child to shift up
            if ((2*i==n-1) or (self.heap[2*i] > self.heap[(2*i)+1])):
              self.heap[i], self.heap[2*i] = self.heap[2*i], self.heap[i]
              i = 2*i
            else:
              self.heap[i], self.heap[(2*i)+1] = self.heap[(2*i)+1], self.heap[i]
              i = (2*i)+1

## 3. Routing algorithms

### 3.1 Dijkstra's Algorithm without Heap

The code below implements Dijkstra's algorithm on a graph without using a heap structure for selecting the element from the fringe to add to the tree.

In [0]:
def Dijkstra_No_Heap(G, s, t):

  #record the start time
  start_time = time.time()
  s, t = s-1, t-1

  #initialize necessary variables
  status = ['unseen'] * 5000
  dad = [0] * 5000
  bw = [0] * 5000
  status[s] = 'intree'
  path = str(t+1)

  #start adding the neighbors of start to fringe
  for v, w in G.getEdges(s):
    status[v] = 'fringe'
    dad[v] = s
    bw[v] = w
  
  while(True):
    u, w = -1, 0

    #Check for the fringe with minimum bw
    for i in range(5000):
      if status[i] == 'fringe' and bw[i] > w:
        w = bw[i]
        u = i

    #Add vertex to tree
    status[u] = 'intree'

    #stop if target is found
    if u == t:
      #trace the path
      tmp = t
      while (tmp!=s):
        tmp = dad[tmp]
        path = str(tmp+1)+'->'+path
      #return the bw, path and time of execution
      return w, path, time.time()-start_time
    
    #Check all the neighbors of the vertex
    for v, w1 in G.getEdges(u):

      #Add all unseen vertices to fringe
      if status[v] == 'unseen':
        status[v] = 'fringe'
        dad[v] = u
        bw[v] = min(w,w1)

      #check the all the fringes if better bw is available
      elif status[v] == 'fringe' and min(w,w1) > bw[v]:
        dad[v] = u
        bw[v] = min(w,w1)

### 3.2 Dijkstra's Algorithm with Heap

The code below implements Dijkstra's algorithm on a graph using a heap structure for selecting the element from the fringe to add to the tree.

In [0]:
def Dijkstra_Heap(G, s, t):

  #record the start time
  start_time = time.time()
  s, t = s-1, t-1

  #initialize necessary variables
  status = ['unseen'] * 5000
  dad = [0] * 5000
  bw = [0] * 5000
  status[s] = 'intree'
  path = str(t+1)
  H = MaxHeap()

  #start adding the neighbors of start to fringe
  for v, w in G.getEdges(s):
    status[v] = 'fringe'
    dad[v] = s
    bw[v] = w
    H.push([w,v])
  
  while(True):

    #Pop the element from heap
    w, u = H.pop()

    #Check if vertex already in tree
    if status[u] == 'intree':
      continue

    #Add vertex to tree
    status[u] = 'intree'

    #stop if target is found
    if u == t:
      #trace the path
      tmp = t
      while (tmp!=s):
        tmp = dad[tmp]
        path = str(tmp+1)+'->'+path
      #return the bw, path and time of execution
      return w, path, time.time()-start_time
    
    #Check all the neighbors of the vertex
    for v, w1 in G.getEdges(u):

      #Add all unseen vertices to fringe
      if status[v] == 'unseen':
        status[v] = 'fringe'
        dad[v] = u
        bw[v] = min(w,w1)
        H.push([min(w,w1),v])

      #check the all the fringes if better bw is available
      elif status[v] == 'fringe' and min(w,w1) > bw[v]:
        dad[v] = u
        bw[v] = min(w,w1)
        H.push([min(w,w1),v])

### 3.3 Kruskal's algorithm

The code below implements Kruskal's algorithm to build a maximum spanning tree and to find the maximum bandwidth path from the tree.

In [0]:
class UnionFind:

  #initialize the dad and wt of tree as dictionaries
  def __init__(self):
        self.wt = {}
        self.dad = {}

  #define getitem
  def __getitem__(self, object):
        #if item is not in dad, add item and return
        if object not in self.dad:
            self.dad[object] = object
            self.wt[object] = 1
            return object

        path = [object]
        root = self.dad[object]
        #trace all ancestors
        while root != path[-1]:
            path.append(root)
            root = self.dad[root]
        #assign dad of all ancestors to be root
        for ancestor in path:
            self.dad[ancestor] = root
        #return root
        return root
        
  def __iter__(self):
        return iter(self.dad)
      
  def union(self, *objects):
        roots = [self[x] for x in objects]
        heaviest = max([(self.wt[r],r) for r in roots])[1]
        for r in roots:
            #find the heaviest tree and add the other tree
            if r != heaviest:
                self.wt[heaviest] += self.wt[r]
                self.dad[r] = heaviest

In [0]:
#define the heapify function
def heapify(arr, n, i): 
    smallest = i  
    l = 2 * i + 1     
    r = 2 * i + 2      
    if l < n and arr[i] > arr[l]: 
        smallest = l 
    if r < n and arr[smallest] > arr[r]: 
        smallest = r 
    if smallest != i: 
        arr[i],arr[smallest] = arr[smallest],arr[i] 
        heapify(arr, n, smallest)

#define the heapsort function
def heapSort(arr): 
    n = len(arr) 
    for i in range(int(n/2)-1, -1, -1): 
        heapify(arr, n, i) 
    for i in range(n-1, 0, -1): 
        arr[i], arr[0] = arr[0], arr[i] 
        heapify(arr, i, 0)

In [0]:
def Kruskal(G):

  #record the start time
  start_time = time.time()
  global visited
  visited = set()

  #get the list of edges
  edges = G.getEdgeList()

  #sort the edges using heapsort
  heapSort(edges)

  #initialize the unionfind object and a new graph for the spanning tree
  subtrees = UnionFind()
  T = Graph()

  #iterate all edges in edgelist
  for w, u, v in edges:

    #check if the two vertices does not belong to the same tree
    if subtrees[u] != subtrees[v]:
      T.InsertEdge(u, v, w)
      subtrees.union(u, v)

  #return the spanning tree
  return T, time.time()-start_time

In [0]:
#define the BFS algorithm to search for the route in Spanning tree
def bfs(G, s, t):

  start_time = time.time()
  s, t = s-1, t-1

  #record the start time
  if s == t:
    return 0, str(s+1)
  
  path = str(s+1)
  Q = deque()
  visited = set([s])
  #add edges of start vertex
  for v, w in G.getEdges(s):
    if v not in visited:
      Q.append([v,w,path+'->'+str(v+1)])
      visited.add(v)
  
  while(True):
    v, w, path = Q.popleft()

    #return if target is found
    if v == t:
      return w, path, time.time()-start_time
    
    for v1, w1 in G.getEdges(v):
      if v1 not in visited:
        Q.append([v1,min(w,w1),path+'->'+str(v1+1)])
        visited.add(v1)

## 4. Testing

I am randomly generating sources and targets from a pool of 5000 values and running the same set of sources and targets on 5 versions of Sparse Graphs and 5 different versions of Dense Graphs. 

In [0]:
pool = list(range(1,5001))
sources = []
targets = []

for _ in range(5):
  source = random.choice(pool)
  pool.remove(source)
  target = random.choice(pool)
  pool.remove(target)
  sources.append(source)
  targets.append(target)

testSet = list(zip(sources,targets))

In [0]:
SparseGraphs = []
DenseGraphs = []
for i in range(5):
  SparseGraphs.append(generateSparseGraph())
  DenseGraphs.append(generateDenseGraph())

### Testing Dijkstra's Algorithm without Heap on Sparse Graph

In [14]:
bw1 = [[0]*5]*5
path1 = [['']*5]*5
time1 = [[0 for _ in range(5)] for _ in range(5)]
i = 0
for SparseGraph in SparseGraphs:
  j = 0
  print("Results for Sparse Graph ",i+1,'\n')
  for source, target in testSet:
    bw1[i][j], path1[i][j], time1[i][j] = Dijkstra_No_Heap(SparseGraph, source, target)
    print("Source: ",source,"\t"+"Target: ",target)
    print("Maximum Bandwidth: ", bw1[i][j])
    print("Path Followed by algotithm: "+path1[i][j])
    print("Time taken by algorithm: "+"%.3f seconds"%time1[i][j],'\n')
    j+=1
  print('\n\n\n')
  i+=1

Results for Sparse Graph  1 

Source:  4747 	Target:  4051
Maximum Bandwidth:  730
Path Followed by algotithm: 4747->3854->3042->379->258->2647->1553->945->212->2225->3215->2231->136->352->2850->482->43->1157->2207->367->2702->832->2678->2750->2939->2896->1792->2780->803->788->1297->835->3301->3->1625->2190->512->573->3374->982->3305->1400->1612->979->2023->1785->3365->2659->1596->676->3725->3277->4051
Time taken by algorithm: 0.649 seconds 

Source:  4942 	Target:  4015
Maximum Bandwidth:  589
Path Followed by algotithm: 4942->1264->2270->3566->2096->3518->4323->2494->569->502->857->376->2396->993->458->205->1512->1559->881->245->1686->849->1044->783->43->1157->1251->3742->2967->4015
Time taken by algorithm: 1.159 seconds 

Source:  3262 	Target:  2191
Maximum Bandwidth:  307
Path Followed by algotithm: 3262->4450->529->1350->457->1304->346->1144->720->1082->633->642->1343->195->487->1227->860->1402->165->1128->1362->1364->1089->520->31->698->764->119->910->584->2704->1484->2996->2191

### Testing Dijkstra's Algorithm with Heap on Sparse Graph

In [15]:
bw2 = [[0]*5]*5
path2 = [['']*5]*5
time2 = [[0 for _ in range(5)] for _ in range(5)]
i = 0
for SparseGraph in SparseGraphs:
  j = 0
  print("Results for Sparse Graph ",i+1,'\n')
  for source, target in testSet:
    bw2[i][j], path2[i][j], time2[i][j] = Dijkstra_Heap(SparseGraph, source, target)
    print("Source: ",source,"\t"+"Target: ",target)
    print("Maximum Bandwidth: ", bw2[i][j])
    print("Path Followed by algotithm: "+path2[i][j])
    print("Time taken by algorithm: "+"%.3f seconds"%time2[i][j],'\n')
    j+=1
  print('\n\n\n')
  i+=1

Results for Sparse Graph  1 

Source:  4747 	Target:  4051
Maximum Bandwidth:  730
Path Followed by algotithm: 4747->3854->4400->4889->4732->3862->4140->1576->2768->2651->4526->4714->3664->1791->2056->1898->1582->3350->3378->4668->2806->4871->4382->3978->2434->1801->4051
Time taken by algorithm: 0.021 seconds 

Source:  4942 	Target:  4015
Maximum Bandwidth:  589
Path Followed by algotithm: 4942->1264->2270->3566->2096->3518->4323->4467->3893->3795->4125->3823->2933->4450->4679->4812->4632->4688->3698->3292->3920->3930->4301->3927->3053->4199->3548->3754->3376->4680->2945->3247->4348->2959->4039->4222->4076->3596->2765->3922->2800->4477->4001->1597->4497->4405->4368->1816->4015
Time taken by algorithm: 0.056 seconds 

Source:  3262 	Target:  2191
Maximum Bandwidth:  307
Path Followed by algotithm: 3262->4450->4679->4812->4632->3558->3853->3497->4656->4868->4121->4143->3925->4270->4073->3922->4762->4647->4500->4395->4001->4477->3960->4323->4426->3907->4434->3873->4933->4813->4393->4253-

### Testing Kruskal's Algorithm on Sparse Graph

In [16]:
bw3 = [[0]*5]*5
path3 = [['']*5]*5
time3 = [[0 for _ in range(5)] for _ in range(5)]
timeG1 = [0 for _ in range(5)]
i = 0
for SparseGraph in SparseGraphs:
  j = 0
  T, timeG1[i] = Kruskal(SparseGraph)
  print("Results for Sparse Graph ",i+1,'\n')
  for source, target in testSet:
    bw3[i][j], path3[i][j], time3[i][j] = bfs(T,source,target)
    print("Source: ",source,"\t"+"Target: ",target)
    print("Maximum Bandwidth: ", bw3[i][j])
    print("Path Followed by algotithm: "+path3[i][j])
    print("Time taken by algorithm: "+"%.3f seconds"%(timeG1[i]+time3[i][j]),'\n')
    j+=1
  print('\n\n\n')
  i+=1

Results for Sparse Graph  1 

Source:  4747 	Target:  4051
Maximum Bandwidth:  730
Path Followed by algotithm: 4747->3854->3042->379->4156->4125->4664->4820->2381->2901->662->2499->2238->470->2388->1535->3099->1007->4466->2213->3177->2240->3751->4245->834->2801->4802->4564->411->2167->756->3468->3445->4658->3628->2105->3212->3451->4460->2720->4467->4323->4426->3580->3526->853->3283->259->2597->310->3495->4639->1875->1035->3034->4406->3128->4714->3664->1791->2056->2945->4680->3564->4791->2850->352->136->2542->1867->3427->3609->1212->3414->2275->1126->3616->4587->3915->1859->1531->2419->3401->3586->2518->3869->3365->2659->1596->676->3725->3277->4051
Time taken by algorithm: 0.213 seconds 

Source:  4942 	Target:  4015
Maximum Bandwidth:  589
Path Followed by algotithm: 4942->1264->2270->3566->2096->3518->4323->4467->2720->4460->3451->3212->2105->3628->4658->3445->3468->756->2167->411->4564->4802->2801->834->909->4244->1527->3653->3556->336->3396->4559->2967->4015
Time taken by algorithm:

### Testing Dijkstra's Algorithm without Heap on Dense Graph

In [17]:
bw4 = [[0]*5]*5
path4 = [['']*5]*5
time4 = [[0 for _ in range(5)] for _ in range(5)]
i = 0
for DenseGraph in DenseGraphs:
  j = 0
  print("Results for Sparse Graph ",i+1,'\n')
  for source, target in testSet:
    bw4[i][j], path4[i][j], time4[i][j] = Dijkstra_No_Heap(DenseGraph, source, target)
    print("Source: ",source,"\t"+"Target: ",target)
    print("Maximum Bandwidth: ", bw4[i][j])
    print("Path Followed by algotithm: "+path4[i][j])
    print("Time taken by algorithm: "+"%.3f seconds"%time4[i][j],'\n')
    j+=1
  print('\n\n\n')
  i+=1

Results for Sparse Graph  1 

Source:  4747 	Target:  4051
Maximum Bandwidth:  998203
Path Followed by algotithm: 4747->4906->1643->2002->741->3099->2079->2734->1531->1512->413->2275->2013->1061->1872->1214->1788->1443->3543->2277->300->1503->2514->2799->3776->3507->381->4160->4051
Time taken by algorithm: 3.837 seconds 

Source:  4942 	Target:  4015
Maximum Bandwidth:  997107
Path Followed by algotithm: 4942->426->349->684->719->1683->672->1940->1289->796->493->441->1535->1253->1397->23->136->1588->1600->143->1201->680->1075->1017->228->85->940->574->578->910->1400->952->774->182->1280->1154->412->442->1372->1278->1483->975->1276->3->1808->1907->287->979->698->2311->1445->2264->642->2760->1045->1992->4015
Time taken by algorithm: 3.230 seconds 

Source:  3262 	Target:  2191
Maximum Bandwidth:  998019
Path Followed by algotithm: 3262->909->347->1950->2075->2884->2386->426->2975->458->412->1715->2190->1761->390->413->1512->1531->3->1808->569->1195->1842->114->2336->2153->267->817->2604-

### Testing Dijkstra's Algorithm with Heap on Dense Graph

In [18]:
bw5 = [[0]*5]*5
path5 = [['']*5]*5
time5 = [[0 for _ in range(5)] for _ in range(5)]
i = 0
for DenseGraph in DenseGraphs:
  j = 0
  print("Results for Sparse Graph ",i+1,'\n')
  for source, target in testSet:
    bw5[i][j], path5[i][j], time5[i][j] = Dijkstra_Heap(DenseGraph, source, target)
    print("Source: ",source,"\t"+"Target: ",target)
    print("Maximum Bandwidth: ", bw5[i][j])
    print("Path Followed by algotithm: "+path5[i][j])
    print("Time taken by algorithm: "+"%.3f seconds"%time5[i][j],'\n')
    j+=1
  print('\n\n\n')
  i+=1

Results for Sparse Graph  1 

Source:  4747 	Target:  4051
Maximum Bandwidth:  998203
Path Followed by algotithm: 4747->4906->1643->2002->4420->3426->2148->4223->4662->4698->4979->4461->2135->3958->1923->2323->3492->2590->2042->1456->2054->736->2315->4795->4573->4268->2514->2799->3776->3507->381->4160->4051
Time taken by algorithm: 2.290 seconds 

Source:  4942 	Target:  4015
Maximum Bandwidth:  997107
Path Followed by algotithm: 4942->426->2975->3731->2365->3179->3739->2654->3261->3940->3809->3545->3632->3299->4748->4009->3862->4508->4544->3620->3823->3984->4155->4185->3711->3677->3695->3486->3829->4399->4255->4557->3236->2502->3755->2760->1045->1992->4015
Time taken by algorithm: 2.227 seconds 

Source:  3262 	Target:  2191
Maximum Bandwidth:  998019
Path Followed by algotithm: 3262->909->4330->2566->3060->3260->4642->3364->2725->2346->4830->4881->3589->4962->4885->3768->3901->4345->3420->3289->3646->3962->4946->2809->3558->3314->3821->2810->2604->2044->1134->2191
Time taken by algor

### Testing Kruskal's Algorithm on Dense Graph

In [37]:
bw6 = [[0]*5]*5
path6 = [['']*5]*5
time6 = [[0 for _ in range(5)] for _ in range(5)]
timeG2 = [0 for _ in range(5)]
i = 0
for DenseGraph in DenseGraphs:
  j = 0
  T, timeG2[i] = Kruskal(DenseGraph)
  print("Results for Sparse Graph ",i+1,'\n')
  for source, target in testSet:
    bw6[i][j], path6[i][j], time6[i][j] = bfs(T,source,target)
    print("Source: ",source,"\t"+"Target: ",target)
    print("Maximum Bandwidth: ", bw6[i][j])
    print("Path Followed by algotithm: "+path6[i][j])
    print("Time taken by algorithm: "+"%.3f seconds"%(timeG2[i]+time6[i][j]),'\n')
    j+=1
  print('\n\n\n')
  i+=1

Results for Sparse Graph  1 

Source:  4747 	Target:  4051
Maximum Bandwidth:  998203
Path Followed by algotithm: 4747->4906->1643->2002->4420->3426->2148->4223->4662->4698->4979->4461->142->4052->1091->2861->1435->3941->2020->4686->2187->2140->1658->4876->1148->2154->2738->691->2725->3364->4642->3260->605->3068->146->3842->3029->2277->300->1503->2514->2799->3776->3507->381->4160->4051
Time taken by algorithm: 45.480 seconds 

Source:  4942 	Target:  4015
Maximum Bandwidth:  997107
Path Followed by algotithm: 4942->426->2975->458->412->1154->3664->2805->4689->3222->1149->360->2855->778->4998->1976->4223->4662->4698->4979->1417->1324->1392->2899->2667->2419->1833->653->2424->708->2753->642->2760->1045->1992->4015
Time taken by algorithm: 45.474 seconds 

Source:  3262 	Target:  2191
Maximum Bandwidth:  998019
Path Followed by algotithm: 3262->909->4113->2615->1967->2091->511->2020->3941->1435->2861->1091->4052->2488->287->979->698->712->4899->3122->1502->2827->4596->2692->541->2191
Time